In [2]:
import pandas as pd
from tqdm import tqdm

## Load Clusters and Alarms

In [2]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")

In [3]:
mob = pd.read_parquet("alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [4]:
adsl = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")

In [5]:
ptn = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [6]:
sdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [7]:
pdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [9]:
mob = mob[["alarm_id", "last_occurrence"]]

In [11]:
adsl = adsl[["alarm_id", "last_occurrence"]]
ptn = ptn[["alarm_id", "last_occurrence"]]
sdh = sdh[["alarm_id", "last_occurrence"]]
pdh = pdh[["alarm_id", "last_occurrence"]]


In [12]:
alarms = pd.concat([mob, adsl, ptn, sdh, pdh])


In [13]:
alarms.to_parquet("alarms datasets/alarms.parquet")

## Clusters Statistics

In [30]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters.parquet")
alarms = pd.read_parquet("alarms datasets/alarms.parquet")

In [31]:
clusters_filtered = clusters[["alarm_id", "cluster_id", "cluster_id2","first_occurrence"]]

In [35]:
clusters_complete = clusters_filtered.merge(alarms, on="alarm_id", how="left")

In [44]:
clusters_complete["first_occurrence"] = pd.to_datetime(clusters_complete["first_occurrence"])

In [54]:
clusters_complete = clusters_complete.dropna(subset=["last_occurrence"])

In [56]:
clusters_complete.to_parquet("real-time clusters/clusters_complete.parquet")

### Clusters Size

In [57]:
clusters_grouped_by_id = clusters_complete.groupby(["cluster_id", "cluster_id2"])

In [72]:
clusters_grouped = clusters_grouped_by_id

In [58]:
clusters_grouped_by_id.size()

cluster_id        cluster_id2     
202301010000_1    202301010000_33     1
202301010000_10   202301010000_133    1
202301010000_104  202301010000_79     1
202301010000_105  202301010000_80     1
202301010000_106  202301010000_81     1
                                     ..
202312312358_5    202312312354_5      1
202312312358_6    202312312354_5      1
202312312358_7    202312312354_5      1
202312312358_8    202312312354_7      1
202312312358_9    202312312354_7      1
Length: 6369371, dtype: int64

### Clusters Lifespan

In [73]:
# Definisci una funzione per calcolare la lifespan
def calculate_lifespan(group):
    return group['last_occurrence'].max() - group['first_occurrence'].min()

tqdm.pandas()

# Raggruppa il dataframe e applica la funzione di calcolo della lifespan
lifespan_df = clusters_grouped.apply(calculate_lifespan, include_groups=False).reset_index(name='lifespan')


In [75]:
lifespan_df.to_parquet("real-time clusters/lifespan_df.parquet")

### Clusters Delta First Occurrence

In [76]:


# Definisci una funzione per calcolare la lifespan
def calculate_lifespan(group):
    return group['first_occurrence'].max() - group['first_occurrence'].min()

# Crea un oggetto tqdm per il DataFrame
tqdm.pandas()

# Raggruppa il dataframe e applica la funzione di calcolo della lifespan
delta_df = clusters_grouped.progress_apply(calculate_lifespan).reset_index(name='lifespan')

100%|██████████| 6369371/6369371 [17:32<00:00, 6048.92it/s]


In [77]:
delta_df

,cluster_id,cluster_id2,lifespan
0,202301010000_1,202301010000_33,0 days
1,202301010000_10,202301010000_133,0 days
2,202301010000_104,202301010000_79,0 days
3,202301010000_105,202301010000_80,0 days
4,202301010000_106,202301010000_81,0 days
...,...,...,...
6369366,202312312358_5,202312312354_5,0 days
6369367,202312312358_6,202312312354_5,0 days
6369368,202312312358_7,202312312354_5,0 days
6369369,202312312358_8,202312312354_7,0 days


In [78]:
delta_df.to_parquet("real-time clusters/delta_df.parquet")